# Efficiency: What percent of a home’s average daily energy use is provided by its on-site solar PV-generated power

To prepare the data we selected 50 Texas homes with PV. We used 1 minute energy(from electricity.eg_realpower_1min) data for the selected homes for March 2018 through August 2018.

In [ ]:
#import packages
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import sqlalchemy as sqla
import os
from config.read_config import get_database_config
from datetime import datetime as dt
import numpy as np
import sys
%matplotlib inline
sys.executable  # shows you your path to the python you're using

In [ ]:
#Read pre-processed data 
data_df = pd.read_csv('/shared/whole-use-by-solar.gz', compression='gzip')
homes_list = data_df.dataid.unique() 
homes_list

In [ ]:
#Function to calculate how much energy use was powered by solar for each home for each interval
def calc_use_powered_by_solar(row):

    if row['grid'] < 0 and row['solar'] > 0:

        val = row['total_use']

    elif row['solar'] > 0 and row['grid'] > 0:

        val = row['solar']

    else:

        val = 0

    return val

In [ ]:
#replace null with 0's 
data_df = data_df.fillna(0)

#create new column use.
data_df['total_use'] = data_df['grid'] + data_df['solar']

data_df['use_powered_by_solar'] = data_df.apply(calc_use_powered_by_solar, axis=1)

data_df.head(10)

In [ ]:
# convert localminute to date
#data_df['dt'] = data_df['localminute'].dt.normalize()

data_df['localminute'] = pd.to_datetime(data_df['localminute'])
data_df['dt'] = data_df['localminute'].dt.date
data_df.head(10)

In [ ]:
#calculate home's daily average use powered by solar
avg_solar_use_by_day = pd.DataFrame(data_df, columns = ['dt','total_use','use_powered_by_solar'])
avg_solar_use_by_day = avg_solar_use_by_day.groupby(['dt']).sum()
avg_solar_use_by_day = avg_solar_use_by_day.reset_index()
avg_solar_use_by_day['use_from_solar'] = (avg_solar_use_by_day['use_powered_by_solar']/avg_solar_use_by_day['total_use'])*100
avg_solar_use_by_day.head(10)

In [ ]:
print(avg_solar_use_by_day['use_from_solar'].mean())

In [ ]:
#calculate home's weekly average use powered by solar
avg_solar_use_by_week = pd.DataFrame(data_df, columns = ['dt','total_use','use_powered_by_solar'])
avg_solar_use_by_week['dt'] = pd.to_datetime(avg_solar_use_by_week['dt'])
avg_solar_use_by_week['week_num'] = avg_solar_use_by_week['dt'].dt.week
#avg_solar_use_by_week['week_num'] = avg_solar_use_by_week['dt'].dt.week
avg_solar_use_by_week = avg_solar_use_by_week.groupby(['week_num']).sum()
avg_solar_use_by_week = avg_solar_use_by_week.reset_index()
avg_solar_use_by_week['use_from_solar'] = (avg_solar_use_by_week['use_powered_by_solar']/avg_solar_use_by_week['total_use'])*100
avg_solar_use_by_week.head(10)




In [ ]:
#plot bar chart for average weekly use
avg_solar_use_by_week.plot.bar(x='week_num', y='use_from_solar',figsize=(20,10),color='orange',title="Homes's average weekly use powered by solar")
plt.show()

## From above analysis we see that 29% of home’s average daily energy use is provided by its on-site solar PV-generated power.